In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
GEN_BACK_TR = True

In [ ]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time
import numpy as np, pandas as pd, seaborn as sns
from multiprocessing import Pool
#nlp augmentation
!pip install --quiet googletrans==3.1.0a0
!pip install --quiet textattack 
from googletrans import Translator

#model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold

#for fast parallel processing
from dask import bag, diagnostics

In [19]:
combined_dataset=pd.read_csv("/content/drive/MyDrive/NLP/data/clean-hindi-train.csv")
combined_dataset = combined_dataset[combined_dataset.notnull()]
combined_dataset['text'] = combined_dataset['text'].str[:100] 

In [20]:
from textattack.augmentation import EmbeddingAugmenter
from textattack.augmentation import WordNetAugmenter
embed_aug = EmbeddingAugmenter()
wordnet_aug = WordNetAugmenter()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [21]:
def back_translate(sequence,label):
    languages = ['en']
    #instantiate translator
    translator = Translator()
    output_sequence = [sequence]
    label_sequence = [label]
    #store original language so we can convert back
    org_lang = 'hi'
    #randomly choose language to translate sequence to  
    for language  in languages:
      #translate to new language and back to original
      translated = translator.translate(sequence, dest = language).text
      
      translated_back1 = translator.translate(wordnet_aug.augment(translated)[0], dest = org_lang).text
      translated_back2 = translator.translate(embed_aug.augment(translated)[0], dest = org_lang).text
      output_sequence.append(translated_back1)
      label_sequence.append(label)
      output_sequence.append(translated_back2)
      label_sequence.append(label)
    
    news_list.extend(output_sequence)
    label_list.extend(label_sequence)


In [22]:
def back_translate_parallel(dataset):
    news = (dataset['text']).tolist()
    labels = (dataset['label']).tolist()
    with diagnostics.ProgressBar():
        for i in range(0,len(news)):
          print(i)
          back_translate(news[i],labels[i])

In [ ]:
news_list = list()
label_list = list()
if True:
#now we apply translation augmentation
    back_translate_parallel(combined_dataset)
    dic = {"text":news_list,"label":label_list}
    df = pd.DataFrame(dic)
    df.to_csv("Augmented_Train.csv",index=False)